In [ ]:
# Import necessary items
from matplotlib import pyplot as plt
from tensorflow import keras
import tensorflow as tf
import cv2
import numpy as np
import pickle
import tarfile
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D, Dense, Flatten, Reshape
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
# Mounting  and opening dataset
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Building list of datasets dir
myimages = [] #list of images 
myannot = [] #list of annotations
dirMain = os.listdir('/content/driver_161_90frame') #list of directory files
for folder in sorted(dirMain):
  counter_jpg=0
  counter_txt=0
  for file in sorted(os.listdir('/content/driver_161_90frame/' + folder)):
      if '.jpg' in file:
          image_name=file.split(".")
          image1_name=image_name[0]
          text1_name = image1_name  + '.lines.txt'
          file1_name=file
          counter_jpg=counter_jpg + 1
          for file in sorted(os.listdir('/content/driver_161_90frame/' + folder)):
             if (text1_name in file):               
               myannot.append('/content/driver_161_90frame/' + folder + '/' + file)
               myimages.append('/content/driver_161_90frame/' + folder + '/' + file1_name)
               counter_txt=counter_txt + 1
############################################################################
# Building images tensor
N_train_image = 10000
train_images = []
from tqdm import tqdm
x_pixels = 590
y_pixels = 1640
train_labels = []
for j in tqdm(range(len(myannot))):
    lines = []
    with open(myannot[j], 'r') as f:
        data = f.readlines()
        if os.stat(myannot[j]).st_size == 0:
          pass
        else:
          lines = [line.split() for line in data]
          lines = [[int(float(i)) for i in line] for line in lines]
          lines = [[(line[i], line[i + 1]) for i in range(0, len(line), 2)] for line in lines]
          lines = [line for line in lines if len(line) != 1]
          new_lines = [*lines]
          lines = []
          for line in new_lines:
              try:
                  x1 = line[0][0]
                  y1 = line[0][1]
                  x2 = line[-1][0]
                  y2 = line[-1][1]

                  m = (y2 - y1) / (x2 - x1)
                  b = y1 - m * x1

                  lines.append((x1, y1, x2, y2, m, b))
              except:
                  continue
          label = np.zeros((5000, 5000))
          for line in lines:
              x1, y1, x2, y2, m, b = line
              x = np.arange(x1, x2, 1 if x2 > x1 else -1)
              y = m * x + b
              y = np.array(y, dtype=int)
              label[y, x] = 255
              # print(tf.reduce_max(label))
          label = label[:590, :1640]
          label = np.array(Image.fromarray(label).resize((328, 118)))
          train_labels.append(label)
          myimages[j] = plt.imread(myimages[j])  #opencv2.resize or pilow
          scale_percent = 50 # percent of original size
          width = int(myimages[j].shape[1] * scale_percent / 100)
          height = int(myimages[j].shape[0] * scale_percent / 100)
          dim = (width, height)      
          myimages[j] = np.array(Image.fromarray(myimages[j]).resize((328, 118)))
          train_images.append(myimages[j])          
    if j == N_train_image-1:
      break
train_labels = np.array(train_labels)
train_labels = np.expand_dims(train_labels, -1)
train_images = np.array(train_images)
train_images = train_images[:,54:,:,:]
train_labels = train_labels[:,54:,:,:]
print(train_labels.shape)
print(train_images.shape)
label_max = tf.reduce_max(train_labels)
train_labels = train_labels/label_max
train_labels = train_images/255
print(tf.reduce_max(train_labels))
print(tf.reduce_max(train_images))

 55%|█████▍    | 9999/18233 [15:00<12:21, 11.11it/s]


(8838, 118, 328, 1)
(8838, 118, 328, 3)


In [ ]:
# Building test data
myimages_test = [] #list of images test
myannot_test = [] #list of annotations test
dirMain = os.listdir('/content/driver_37_30frame') #list of directory files
for folder in sorted(dirMain):
  counter_jpg=0
  counter_txt=0
  for file in sorted(os.listdir('/content/driver_37_30frame/' + folder)):
      if '.jpg' in file:
          image_name=file.split(".")
          image1_name=image_name[0]
          text1_name = image1_name  + '.lines.txt'
          file1_name=file
          counter_jpg=counter_jpg + 1
          for file in sorted(os.listdir('/content/driver_37_30frame/' + folder)):
             if (text1_name in file):               
               myannot_test.append('/content/driver_37_30frame/' + folder + '/' + file)
               myimages_test.append('/content/driver_37_30frame/' + folder + '/' + file1_name)
############################################################################
# Building test image and annot tensor
N_test_image = 500
test_images = []
from tqdm import tqdm
x_pixels = 590
y_pixels = 1640
test_labels = []
for j in tqdm(range(len(myannot_test))):
    lines = []
    with open(myannot_test[j], 'r') as f:
        data = f.readlines()
        if os.stat(myannot_test[j]).st_size == 0:
          pass
        else:
          lines = [line.split() for line in data]
          lines = [[int(float(i)) for i in line] for line in lines]
          lines = [[(line[i], line[i + 1]) for i in range(0, len(line), 2)] for line in lines]
          lines = [line for line in lines if len(line) != 1]
          new_lines = [*lines]
          lines = []
          for line in new_lines:
              try:
                  x1 = line[0][0]
                  y1 = line[0][1]
                  x2 = line[-1][0]
                  y2 = line[-1][1]

                  m = (y2 - y1) / (x2 - x1)
                  b = y1 - m * x1

                  lines.append((x1, y1, x2, y2, m, b))
              except:
                  continue
          label = np.zeros((5000, 5000))
          for line in lines:
              x1, y1, x2, y2, m, b = line
              x = np.arange(x1, x2, 1 if x2 > x1 else -1)
              y = m * x + b
              y = np.array(y, dtype=int)
              label[y, x] = 255
          label = label[:590, :1640]
          label = np.array(Image.fromarray(label).resize((328, 118)))
          test_labels.append(label)
          myimages_test[j] = plt.imread(myimages_test[j])  #opencv2.resize or pilow
          scale_percent = 50 # percent of original size
          width = int(myimages_test[j].shape[1] * scale_percent / 100)
          height = int(myimages_test[j].shape[0] * scale_percent / 100)
          dim = (width, height)      
          myimages_test[j] = np.array(Image.fromarray(myimages_test[j]).resize((328, 118)))
          test_images.append(myimages_test[j])          
    if j == N_test_image-1:
      break
test_labels = np.array(test_labels)
test_labels = np.expand_dims(test_labels, -1)
test_images = np.array(test_images)
test_images = test_images[:,54:,:,:]
test_labels = test_labels[:,54:,:,:]
print(test_labels.shape)
print(test_images.shape)
label_max = tf.reduce_max(test_labels)
test_labels = test_labels/label_max
test_labels = test_images/255
print(tf.reduce_max(test_labels))
print(tf.reduce_max(test_images))

In [ ]:
def create_model1(input_shape, pool_size):
    # Create the actual neural network here
    model = Sequential()
    # Normalizes incoming inputs. First layer needs the input shape to work
    model.add(BatchNormalization(input_shape=input_shape))

    # Below layers were re-named for easier reading of model summary; this not necessary
    # Conv Layer 1
    model.add(Conv2D(8, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Conv1'))

    # Conv Layer 2
    model.add(Conv2D(16, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Conv2'))

    # Pooling 1
    model.add(MaxPooling2D(pool_size=pool_size))

    # Conv Layer 3
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Conv4'))
    model.add(Dropout(0.2))

    # Conv Layer 4
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Conv5'))
    model.add(Dropout(0.2))

    # Conv Layer 5
    model.add(Conv2D(64, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Conv6'))
    model.add(Dropout(0.2))

    # Conv Layer 6
    model.add(Conv2D(64, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Conv7'))
    model.add(Dropout(0.2))

    # Upsample 1
    model.add(UpSampling2D(size=pool_size))

    # Deconv 1
    model.add(Conv2DTranspose(64, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Deconv1'))
    model.add(Dropout(0.2))

    # Deconv 2
    model.add(Conv2DTranspose(64, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Deconv2'))
    model.add(Dropout(0.2))

    # Deconv 3
    model.add(Conv2DTranspose(32, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Deconv3'))
    model.add(Dropout(0.2))

    # Deconv 4
    model.add(Conv2DTranspose(32, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Deconv4'))
    model.add(Dropout(0.2))

    # Deconv 6
    model.add(Conv2DTranspose(16, (3, 3), padding='same', strides=(1,1), activation = 'relu', name = 'Deconv6'))

    # Final layer 
    model.add(Conv2DTranspose(1, (3, 3), padding='same', strides=(1,1), activation = 'sigmoid', name = 'Final'))

    return model

In [ ]:
#def main():

# Make into arrays as the neural network wants these
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Shuffle images along with their labels, then split into training/validation sets
train_images, train_labels = shuffle(train_images, train_labels)

# Test size may be 10% or 5%
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.05)

# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 128
epochs = 10
pool_size = (2, 2)
input_shape = X_train.shape[1:]
# Create the neural network
model = create_model1(input_shape, pool_size)

# Using a generator to help the model use less data
# Channel shifts help with shadows slightly
datagen = ImageDataGenerator(channel_shift_range=0.2)
datagen.fit(X_train)

# Compiling and training the model
opt = keras.optimizers.SGD(learning_rate=0.0005)
model.compile(optimizer='Adam', loss='bce', metrics='accuracy')

history = model.fit(X_train, y_train, epochs=epochs, verbose=1, validation_data=(X_val, y_val)) 
# Freeze layers since training is done
model.trainable = False
model.compile(optimizer='Adam', loss='bce')

# Show summary of model
model.summary()

hist = history.history

In [ ]:
score = model.evaluate(test_images,test_labels, verbose=1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    plt.plot(range(1, len(hist['loss']) + 1), hist['loss'],label='train error')
    plt.plot(range(1, len(hist['loss']) + 1), hist['val_loss'], label='val error')
    plt.legend()
plot_history()

In [ ]:
idx = 450
print(np.expand_dims(test_images[idx], 0).shape)
prediction = model.predict(np.expand_dims(test_images[idx], 0)).squeeze()
prediction[prediction < 0.05] = 0
prediction[prediction > 0.05] = 1
plt.figure(dpi=150)
plt.imshow(test_images[idx]+ test_labels[idx])
plt.show()
plt.figure(dpi=150)
plt.imshow(test_images[idx] + np.expand_dims(prediction, -1)*5)
plt.show()
plt.figure(dpi=150)
plt.imshow(prediction+ test_labels[idx].squeeze()) 
plt.show()